In [11]:
import pandas as pd
import re
import hashlib
import datetime

In [60]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [61]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')

'SMARTROLL.settings'

In [62]:
django.setup()

In [63]:
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student
from django.contrib.auth import get_user_model

In [64]:
User = get_user_model()

In [65]:
college_pattern = r"(?P<college_name>[\w\s\.]+)-(?P<college_code>\d+)"
term_pattern = r"(?P<start_year>\d{4})-(?P<end_year>\d{4})"
branch_pattern = r"(?P<branch_name>[\w\s]+)-(?P<branch_code>\d{2})"
college_info = {}
term_info = {}
branch_info = {}

In [66]:
df = pd.read_csv('smartroll.csv')
header_rows = df.iloc[:3].reset_index(drop=True)
for i, value in enumerate(header_rows.iloc[:, 0]):
    if i == 0:        
        match = re.search(college_pattern, value)
        if match:
            college_info["college_name"] = match.group("college_name").strip()
            college_info["college_code"] = match.group("college_code")
    
    elif i == 1:        
        match = re.search(term_pattern, value)
        if match:
            term_info["start_year"] = match.group("start_year")
            term_info["end_year"] = match.group("end_year")
    
    elif i == 2:        
        match = re.search(branch_pattern, value)
        if match:
            branch_info["branch_name"] = match.group("branch_name").strip()
            branch_info["branch_code"] = match.group("branch_code")

# Get the college,Term, Branch Objects
college_obj = College.objects.filter(code=college_info['college_code']).first()
term_obj = Term.objects.filter(start_year=term_info['start_year'],end_year=term_info['end_year']).first()
branch_obj = Branch.objects.filter(branch_code=branch_info['branch_code']).first()
print(college_obj)
print(term_obj)
print(branch_obj)

L.D. College of Engineering
Term - 2024 | 2025
Computer Engineering | Term - 2024 | 2025


In [72]:
df = pd.read_csv('smartroll.csv')
header_rows = df.iloc[:3].reset_index(drop=True)
df.columns = df.iloc[3]
df = df[4:].reset_index(drop=True) 
faculties = list(df.columns[2:])
# Check the format of the faculties row
for faculty in faculties:
    if not faculty or len(faculty) == 0:raise Exception("Formating error - No Faculty Provided")

In [31]:
BE_PATTERN = re.compile(r"^(BE)-(\d+)-([A-Z])(?:_([A-Z]\d))?-([A-Z]+(?:-\w+)?)-(\w+)$")
ME_PATTERN = re.compile(r"^(ME)-(\d+)-([A-Z]+(?:-\w+)?)-(\d+)$")
TIME_PATTERN = re.compile(r"^(\d{2}:\d{2})-(\d{2}:\d{2})$")

In [73]:
def parse_be_me_string(string):
    # Try BE pattern first
    be_match = BE_PATTERN.match(string)
    if be_match:
        course, sem, div, batch, sub, classroom = be_match.groups()
        return {
            "stream": course,
            "sem": sem,
            "div": div,
            "batch": batch if batch else None,  # If no batch, set to None
            "sub": sub,
            "classroom": classroom
        }
    
    # Try ME pattern
    me_match = ME_PATTERN.match(string)
    if me_match:
        course, sem, sub, classroom = me_match.groups()
        return {
            "stream": course,
            "sem": sem,
            "sub": sub,
            "classroom": classroom
        }    
    # If no pattern matches
    return None

def parse_time_string(time_string):
    time_match = TIME_PATTERN.match(time_string)
    if time_match:
        start_time, end_time = time_match.groups()
        return {"start_time": start_time, "end_time": end_time}    
    return None

def hash_string(input_string):
    # Convert the string to bytes
    byte_string = input_string.encode()
    # Create a SHA-256 hash of the byte string
    hash_object = hashlib.sha256(byte_string)
    # Return the hex representation of the hash
    return hash_object.hexdigest()

def check_for_batch_includance(subject_obj,batches):    
    allowed_batches = []
    for batch in batches:
        if subject_obj.included_batches.contains(batch):
            allowed_batches.append(batch)
    return allowed_batches

In [ ]:
for day, day_data in df.groupby('DAY'):
    print(f"Schedule for {day}:")
    rows = list(day_data.iterrows())
    i = 0
    while i < len(rows):
        index, row = rows[i]        
        for faculty in faculties:   
            lecture_type = 'theory'
            time_info = parse_time_string(row['TIME'])
            if time_info:
                info = parse_be_me_string(row[faculty]) if pd.notna(row[faculty]) else None                
                if pd.notna(row[faculty]):
                    current_lecture_hash = hash_string(row[faculty])
                    # Also check if next consecutive lecture is not the same as current
                    if i < len(rows) - 1: 
                        next_row = rows[i+1][1]
                        next_time_info = parse_time_string(next_row['TIME'])
                        next_info = parse_be_me_string(next_row[faculty]) if pd.notna(next_row[faculty]) else None
                        if pd.notna(next_row[faculty]):
                            next_lecture_hash = hash_string(next_row[faculty])
                            if next_lecture_hash == current_lecture_hash:
                                time_info['end_time'] = next_time_info['end_time']
                                rows[i+1][1][faculty] = None          
                                lecture_type = 'lab'
                # Now we can store the lecture data
                if info:                    
                    # Initialize the requirements dictionary object
                    requirements = {
                        'stream':info['stream'],
                        'teacher':Teacher.objects.filter(teacher_code=faculty).first(),
                        'classroom':branch_obj.classroom_set.filter(class_name=info['classroom']).first(),
                        'schedule':None,
                        'time_info':time_info,
                        'type':lecture_type,
                        'subject':None,
                        'batches':None
                    }                                                                                
                    # Get the stream object
                    stream_obj = branch_obj.stream_set.filter(title=info['stream']).first()
                    if stream_obj:
                        # Get the semester                        
                        semester_obj = stream_obj.semester_set.filter(no=info['sem']).first()
                        if semester_obj:
                            # Get the subject
                            requirements['subject'] = semester_obj.subject_set.filter(short_name=info['sub']).first()
                            # get the division
                            # For ME there will be only 1 division per semester
                            if info['stream'] == 'ME':
                                division_obj = semester_obj.division_set.first()
                                # Now get the timetable object of this division
                                timetable_object = division_obj.timetable_set.first()
                                # Get the schedule Object of current day
                                requirements['schedule'] = timetable_object.schedule_set.filter(day=day).first()
                                #  Get the first batch object of the division
                                division_batches = division_obj.batch_set.all()
                                #  check if the batch is included in the subject list or not                                
                                requirements['batches'] = check_for_batch_includance(requirements['subject'],division_batches)                              

                            elif info['stream'] == 'BE':
                                division_obj = semester_obj.division_set.filter(division_name=info['div']).first()
                                # Now get the timetable object of this division
                                timetable_object = division_obj.timetable_set.first()                                
                                # Get the schedule Object of current day
                                requirements['schedule'] = timetable_object.schedule_set.filter(day=day).first()                                
                                # Get the batch
                                # We have to check if the lecture is for the whole class or single batch
                                if info['batch']:
                                    # Now we can get the batch
                                    requirements['batches'] = division_obj.batch_set.filter(batch_name=info['batch'])                                                                                                            
                                    
                                else:
                                    division_batches = division_obj.batch_set.all()                
                                    #  check if the batches are included in the subject list or not
                                    requirements['batches'] = check_for_batch_includance(requirements['subject'],division_batches)                            
                            else:
                                print(info['stream'])
                            # Now we can add the lectures
                            lecture_obj,created = Lecture.objects.get_or_create(start_time=requirements['time_info']['start_time'],end_time=requirements['time_info']['end_time'],schedule=requirements['schedule'],subject=requirements['subject'])
                            if created:
                                lecture_obj.type=requirements['type']                                
                                lecture_obj.teacher=requirements['teacher']
                                lecture_obj.classroom=requirements['classroom']
                                lecture_obj.save()                                
                                lecture_obj.batches.add(*requirements['batches'])
                                # Need to create lecture sessions for this particular lecture...after that the cronjob will take care of it
                                today = datetime.datetime.now().date()                    
                                if lecture_obj:
                                        batches = lecture_obj.batches.all()
                                        lecture_session,created = Session.objects.get_or_create(lecture=lecture_obj,day=today,active='pre')
                                        if created:             
                                            students = Student.objects.filter(batch__in=batches)
                                            for student in students:
                                                attendance_obj = Attendance.objects.create(student=student)
                                                lecture_session.attendances.add(attendance_obj)
                                else:
                                    print('Lecture does not exists')                                
                            else:
                                print(lecture_obj)
                                print('Lecture already exists for this timeslot')                           
        i += 1
    print()    

In [7]:
import requests

url = "https://syllabus.gtu.ac.in/Syllabus.aspx?tp=BE"

payload = '__EVENTTARGET=&__EVENTARGUMENT=&__LASTFOCUS=&__VIEWSTATE=%2FwEPDwUJOTczODk2OTU3D2QWAgIDD2QWDAIBDxAPFgYeDURhdGFUZXh0RmllbGQFC1Byb2dyYW1uYW1lHg5EYXRhVmFsdWVGaWVsZAUGZXh0eXBlHgtfIURhdGFCb3VuZGdkEBUnDVNlbGVjdCBjb3Vyc2UcQXBwbGllZCBTY2llbmNlLSBJQi9DUy9EQi9JTSNCYWNoZWxvciBvZiBFbmdpbmVlcmluZyAoUGFydCBUaW1lKRtCQUNIRUxPUiBPRiBJTlRFUklPUiBERVNJR04UQmFjaGVsb3Igb2YgUGxhbm5pbmcFQkFSQ0gDQkJBA0JDQQdCZGVzaWduAkJFBUJITUNUBkJQSEFSTQNCUFAEQlZvYwdESVBMT01BC0RpcGxvbWEgYXJjBkRQSEFSTQREVm9jBElNQkEITS4gQXJjaC4ETUFIUxJNYXN0ZXIgb2YgUGxhbm5pbmcDTUJBD01CQSAocGFydCB0aW1lKQ5NQkEgSW50ZWdyYXRlZANNQ0EOTUNBIEludGVncmF0ZWQCTUUGTVBIQVJNBU1QSElMBFBEREMFUEdERE0FUEdERFMFUEdESE0GUEdESVBSBlBoYXJtRApQaGFybUQoUEIpA1BIRAdTY2llbmNlFScNU2VsZWN0IGNvdXJzZQtJQi9DUy9EQi9JTQJFUAJCSQJCTAJCQQJCQgJCQwJCTgJCRQJCSAJCUAJQUAJCVgJESQJEQQJEUAJEVgJNQQJNUgJNSAJNTgJNQgJNVgJNQQJNQwJJQwJNRQJNUAJNTAJQRAJETQJEUwJESAJQUgJGRAJQQgJQSAJCUxQrAydnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2cWAQIJZAIDDxAPFgYfAAUHYnJfbmFtZR8BBQdicl9jb2RlHwJnZBAVTQ1TZWxlY3QgQnJhbmNoHTAxIC0gQUVST05BVVRJQ0FMIEVOR0lORUVSSU5HGzAyIC0gQVVUT01PQklMRSBFTkdJTkVFUklORxwwMyAtIEJJTy1NRURJQ0FMIEVOR0lORUVSSU5HEzA0IC0gQklPLVRFQ0hOT0xPR1kZMDUgLSBDSEVNSUNBTCBFTkdJTkVFUklORxYwNiAtIENJVklMIEVOR0lORUVSSU5HGTA3IC0gQ09NUFVURVIgRU5HSU5FRVJJTkcpMDggLSBFTEVDVFJJQ0FMICYgRUxFQ1RST05JQ1MgRU5HSU5FRVJJTkcbMDkgLSBFTEVDVFJJQ0FMIEVOR0lORUVSSU5HHDEwIC0gRUxFQ1RST05JQ1MgRU5HSU5FRVJJTkcsMTEgLSBFTEVDVFJPTklDUyAmIENPTU1VTklDQVRJT04gRU5HSU5FRVJJTkcwMTIgLSBFTEVDVFJPTklDUyAmIFRFTEVDT01NVU5JQ0FUSU9OIEVOR0lORUVSSU5HHjEzIC0gRU5WSVJPTk1FTlRBTCBFTkdJTkVFUklORx8xNCAtIEZPT0QgUFJPQ0VTU0lORyBURUNITk9MT0dZGzE1IC0gSU5EVVNUUklBTCBFTkdJTkVFUklORxsxNiAtIElORk9STUFUSU9OIFRFQ0hOT0xPR1kqMTcgLSBJTlNUUlVNRU5UQVRJT04gJiBDT05UUk9MIEVOR0lORUVSSU5HFzE4IC0gTUFSSU5FIEVOR0lORUVSSU5HGzE5IC0gTUVDSEFOSUNBTCBFTkdJTkVFUklORx0yMCAtIE1FQ0hBVFJPTklDUyBFTkdJTkVFUklORxsyMSAtIE1FVEFMTFVSR1kgRU5HSU5FRVJJTkcXMjIgLSBNSU5JTkcgRU5HSU5FRVJJTkcXMjMgLSBQTEFTVElDIFRFQ0hOT0xPR1kWMjQgLSBQT1dFUiBFTEVDVFJPTklDUxsyNSAtIFBST0RVQ1RJT04gRU5HSU5FRVJJTkcWMjYgLSBSVUJCRVIgVEVDSE5PTE9HWRcyOCAtIFRFWFRJTEUgUFJPQ0VTU0lORxcyOSAtIFRFWFRJTEUgVEVDSE5PTE9HWSMzMSAtIENPTVBVVEVSIFNDSUVOQ0UgJiBFTkdJTkVFUklORyszMiAtIElORk9STUFUSU9OICYgQ09NTVVOSUNBVElPTiBURUNITk9MT0dZHjM0IC0gTUFOVUZBQ1RVUklORyBFTkdJTkVFUklORyczNSAtIEVOVklST05NRU5UQUwgU0NJRU5DRSAmIFRFQ0hOT0xPR1kYMzYgLSBDSEVNSUNBTCBURUNITk9MT0dZKjM3IC0gRU5WSVJPTk1FTlRBTCBTQ0lFTkNFIEFORCBFTkdJTkVFUklORxQzOSAtIE5BTk8gVEVDSE5PTE9HWSc0MCAtIENJVklMICYgSU5GUkFTVFJVQ1RVUkUgRU5HSU5FRVJJTkccNDEgLSBST0JPVElDUyBBTkQgQVVUT01BVElPTlI0MiAtIENPTVBVVEVSIFNDSUVOQ0UgJiBFTkdJTkVFUklORyAoQVJUSUZJQ0lBTCBJTlRFTExJR0VOQ0UgQU5EIE1BQ0hJTkUgTEVBUk5JTkcpLTQzIC0gQVJUSUZJQ0lBTCBJTlRFTExJR0VOQ0UgQU5EIERBVEEgU0NJRU5DRUk0NCAtIENIRU1JQ0FMIEVOR0lORUVSSU5HIChHUkVFTiBURUNITk9MT0dZICYgU1VTVEFJTkFCSUxJVFkgRU5HSU5FRVJJTkcpbDQ1IC0gQ09NUFVURVIgU0NJRU5DRSAmIEVOR0lORUVSSU5HIChJTlRFUk5FVCBPRiBUSElOR1MgQU5EIENZQkVSIFNFQ1VSSVRZIElOQ0xVRElORyBCTE9DSyBDSEFJTiBURUNITk9MT0dZKTI0NiAtIENPTVBVVEVSIFNDSUVOQ0UgJiBFTkdJTkVFUklORyAoREFUQSBTQ0lFTkNFKS40NyAtIEVMRUNUUk9OSUNTICYgSU5TVFJVTUVOVEFUSU9OIEVOR0lORUVSSU5HNDQ4IC0gQ09NUFVURVIgU0NJRU5DRSAmIEVOR0lORUVSSU5HIChDWUJFUiBTRUNVUklUWSkeNDkgLSBDT01QVVRFUiBTQ0lFTkNFICYgREVTSUdOHzUwIC0gU01BUlQgJiBTVVNUQUlOQUJMRSBFTkVSR1kiNTEgLSBGT09EIEVOR0lORUVSSU5HICYgVEVDSE5PTE9HWTE1MiAtIEFSVElGSUNJQUwgSU5URUxMSUdFTkNFIEFORCBNQUNISU5FIExFQVJOSU5HGTUzIC0gUExBU1RJQ1MgRU5HSU5FRVJJTkdFNTQgLSBFTEVDVFJPTklDUyBBTkQgQ09NTVVOSUNBVElPTiAoQ09NTVVOSUNBVElPTiBTWVNURU0gRU5HSU5FRVJJTkcpGzg5IC0gTUVDSEFOSUNBTCBFTkdJTkVFUklORyxBQSAtIE1JTk9SL0hPTk9SUyAtSU5EVVNUUklBTCBQUk9DRVNTIFNBRkVUWSNBQiAtIE1JTk9SL0hPTk9SUyAtV0FTVEUgVEVDSE5PTE9HWSpBQyAtIE1JTk9SL0hPTk9SUyAtQ09OU1RSVUNUSU9OIFRFQ0hOT0xPR1kwQUQgLSBNSU5PUi9IT05PUlMgLU5FWFQgR0VORVJBVElPTiBTTUFSVCBWSUxMQUdFLUFFIC0gTUlOT1IvSE9OT1JTIC1JTkZSQVNUUlVDVFVSRSBFTkdJTkVFUklORx9BRiAtIE1JTk9SL0hPTk9SUyAtU01BUlQgQ0lUSUVTPUFHIC0gTUlOT1IvSE9OT1JTIC1BUlRJRklDSUFMIElOVEVMTElHRU5DRSAmIE1BQ0hJTkUgTEVBUk5JTkchQUggLSBNSU5PUi9IT05PUlMgLUNZQkVSIFNFQ1VSSVRZJUFJIC0gTUlOT1IvSE9OT1JTIC1JTlRFUk5FVCBPRiBUSElOR1MoQUogLSBNSU5PUi9IT05PUlMgLSBTT0xBUiBFTkVSR1kgU1lTVEVNUypBSyAtIE1JTk9SL0hPTk9SUyAtRUxFQ1RSSUNBTCBBTkQgQ09NUFVURVIkQUwgLSBNSU5PUi9IT05PUlMgLUVMRUNUUklDIFZFSElDTEVTOUFNIC0gTUlOT1IvSE9OT1JTIC1DT05UUk9MIFNZU1RFTVMgQU5EIFNFTlNPUlMgVEVDSE5PTE9HWR9BTiAtIE1JTk9SL0hPTk9SUyAtMyBEIFBSSU5USU5HG0FPIC0gTUlOT1IvSE9OT1JTIC1ST0JPVElDUyVBUCAtIE1JTk9SL0hPTk9SUyAtRU5FUkdZIEVOR0lORUVSSU5HPkFRIC0gTUlOT1IvSE9OT1JTIC0gR0xPQkFMIENJVElaRU5TSElQIFBFUlNPTkFMSVRZIERFVkVMT1BNRU5UTEFSIC0gTUlOT1IvSE9OT1VSUy1JTkRVU1RSSUFMIEJBU0VEIE5PTiBERVNUUlVDVElWRSBURUNITklRVUVTIEFORCBQUkFDVElDRVNCQVMgLSBNSU5PUi9IT05PVVJTLUdSRUVOIFRFQ0hOT0xPR1kgQU5EIFNVU1RBSU5BQklMSVRZIEVOR0lORUVSSU5HH0FUIC0gTUlOT1IvSE9OT1VSUy1EQVRBIFNDSUVOQ0UyQVUgLSBNSU5PUi9IT05PVVJTLUZPUkVOU0lDIFNUUlVDVFVSQUwgRU5HSU5FRVJJTkc9QVYgLSBNSU5PUi9IT05PVVJTLUNPTVBVVEVSIEFJREVEIENJVklMIEVOR0lORUVSSU5HIFBST0NFU1NFUwlOQSAtIEFSTVkOTkIgLSBBSVIgRk9SQ0UJTkMgLSBOQVZZFU0NU2VsZWN0IEJyYW5jaAIwMQIwMgIwMwIwNAIwNQIwNgIwNwIwOAIwOQIxMAIxMQIxMgIxMwIxNAIxNQIxNgIxNwIxOAIxOQIyMAIyMQIyMgIyMwIyNAIyNQIyNgIyOAIyOQIzMQIzMgIzNAIzNQIzNgIzNwIzOQI0MAI0MQI0MgI0MwI0NAI0NQI0NgI0NwI0OAI0OQI1MAI1MQI1MgI1MwI1NAI4OQJBQQJBQgJBQwJBRAJBRQJBRgJBRwJBSAJBSQJBSgJBSwJBTAJBTQJBTgJBTwJBUAJBUQJBUgJBUwJBVAJBVQJBVgJOQQJOQgJOQxQrA01nZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2RkAgUPEA8WBh8ABQNzZW0fAQUDc2VtHwJnZBAVCQNTZW0BMQEyATMBNAE1ATYBNwE4FQkDU2VtATEBMgEzATQBNQE2ATcBOBQrAwlnZ2dnZ2dnZ2dkZAIHDxAPFgYfAAUCQVkfAQUCQVkfAmdkEBUODUFjYWRlbWljIFllYXIHMjAyNC0yNQQyMDI0BzIwMjMtMjQHMjAyMi0yMwcyMDIxLTIyBzIwMjAtMjEHMjAxOC0xOQcyMDE3LTE4BzIwMTYtMTcHMjAxNS0xNgcyMDE0LTE1BzIwMTMtMTQHMjAwOC0wORUODUFjYWRlbWljIFllYXIHMjAyNC0yNQQyMDI0BzIwMjMtMjQHMjAyMi0yMwcyMDIxLTIyBzIwMjAtMjEHMjAxOC0xOQcyMDE3LTE4BzIwMTYtMTcHMjAxNS0xNgcyMDE0LTE1BzIwMTMtMTQHMjAwOC0wORQrAw5nZ2dnZ2dnZ2dnZ2dnZ2RkAhcPFgIeB1Zpc2libGVoZAIZD2QWAgIBDzwrABECARAWABYAFgAMFCsAAGQYAQUSR3JpZFZpZXdUb0NhdGVnb3J5D2dkFupyFSlWpDzwkcXMUJbjETEqY6efZFrJ8A8X4FCkUj8%3D&__VIEWSTATEGENERATOR=8F41EED1&__EVENTVALIDATION=%2FwEdAI8BLRDD%2F5N%2FDasptFrG9rzvHEzUfY29B3Neo208CPUkD2ERJK%2BZGosHeTx%2FrNuYmL49Azf3yrwNdUHcuKVtpitSBexROLkZiXN1DVxx7fRB28kIJ6r09cMwbvTX8uh1U4fKL6U3Cy5FFPg9imDJU1Am%2F3mQZm1LY2ze7Gk3jOBMjBw%2FvKImaG9xhmvsuGn7Nq6ElVLjHdYzrVvLvMM8MJ5zqwrQEw2yqO%2BW0CsEURdxq18MNA2b%2FXCDnnB85fyqpWN9x1ZPLADmQYFR64xVRGWR%2BJVcEoVO6nggSE1x0Ge79G5ZU5BhPIIGOksEl61mRIEwU9B8%2FglQNf1NS7%2BeAD5Dws%2Fe2TfSKA27HAeXCV0F%2BU1sPoaxBwYjA2I18pRcN1Esw00g8OjIbojvwa9gvkDfLU3jk1UqW25G%2BTdYo8NGChRK0v5qpZW6Dx6%2B%2BPI0wcvNDU96Zi1lr8jZB3kC20c1jHEwtmIdqiZLnX%2BO3hi9qAB%2B1jqeMml1ElR3XwjDk6XQXzt4fMMReAagMWztHA9yAR9o9Bm1Is2WO5nYNz1sRIlALIlsCibl%2BSSlUR7zURabIrQTzqhmKcQhxRgzwAFgO1VrdoAXrsUL8679fau6qPWwRELVFR1T98fxslBmiMJCKcRNHqPHaC8Umxhq4y3j501LPT%2FGUbzduOwy8nA%2BeFbnDbQCyHgk%2FBUNgNVa3sLhc1hTIBADrva8Eh76XrTbjfJ9w1N%2F8j1XEig8aHbnyJ1vqQED0YKGkRp5%2FaBRGJiiOPI7u1ScaOrKik7G3%2FDkNgn%2BAF1F1VWeFF%2BITfUi%2FHxsXyFrlRhfTUXpECs%2BpkuL0I1kHJ4aipKURo0h9M6oaUvtmPz0BySPScGo%2BxpS3auHF8wTfZcEM%2FTh0wQwJvq%2BRIL%2FuUd%2FllCxXkV1PXshyJnzO1npL1aFR4LdYFN7flhA722rZgXWlniqi4pQPNwmdaJ4goswP6hqoaL5SprnIo%2BjG9LLOPtOBeyOefryw5Qy%2BIr4nB%2BfbN4LrKM2YwL4fgwMG3mjuUr2mfd2%2Fhl6BDreQYkPNrkeICzdHX18ORKNzccS8mSMWqCiwzefo2meSOBwK4MuHc3HLaGEKznjOH2gEhSZMZsDAdokCF16LFc50e82Jbrui6aMr4mG%2FdKGz8Hmb8LoprD3VcG6zdVadGIVvNHtX57KboSyLsWV5ibz1nykjAPRm2HNbK2O4FpsePpyfVIzePnrwjSxCCz6TlaCp4PJBWB8a21NgujwG%2FDxBQpnaQTtnFQYNpy6Hi7Gk2LmuLRwVS1IerRX4lqVxhIUiAai55L1qTEnA2h5JqjjUYT2PRg9H9XnUtHXiOZ5fjVNuLbt0bp9IJG%2BgDR6MGljS7aTsPRRUA4sFRMkkrY6NzK6HqmG7A790nNJsJQVziJfeg%2FBPqXeNctXCN%2F270Ap9yM%2B3IbEAgeRuLIy7M4BtwMUXeTQhQYg2aDOmhN7Dz6IEGHxJXktz72q7V5jvE0kpuwGjmZYd4VX51A1h9kfH9cU%2By9aFSdr%2Fb8PZEm8cohLpYIWX6%2F2W460XrLsGUlIOw8tKzVK%2FjmCDnGuHe4TCr3lN5xUDg3utj2yciyliffHRDxSQrMkMQQkcsTkZABqcwsEnzs3Ctri35Xid4D%2BuRZgfW2Lm0%2Ft9KAYbnI5KsCHDMcYCdFrj6ZNmrs1SX0oAECIaStRKsgV3%2Bfm%2FbifviuIpVHxxZR%2BWJrD%2B4heFKB4Xf7kxKitTm%2BhJZEdy8l6LKT%2Btr%2BG3PcnD0Ofx%2FuPNYVuwBKgNxgRCSnk2WWsO%2FrJ%2FRdnzELzHYV2zSq2Z7i13cLW9JzW5uv5xFnKrBTMAjDVySTGyaF%2F3PaemshsNqypiIABSHfPHK0TIUFT%2BD5IlnicJ8bELOACQ%2F5RfwQ5HrvfZdhl5J%2BbZ%2BPisubiRigY3ubf6kDBBeQ3QixnfXSF2IlO7z9X7Dxtp%2FaA%2B%2BKdfLkdTwPC0YUwB5POm4UQyT6V47uiaGvnwsRJHw%2BmErRlsVti2qSrHqO8amHKXHBvq3BVZWuk4pJOdEYZT4dyoT%2BxBMWEKUY%2FLe8Ook9HWSmhThSarpFkepXM%2BdcZIZwkSgCPc%2FLOpkLYhvAwgo1%2FyKQj%2FUzohC1QqE25pl5nkl8xaeK5IiJbrJNzjT0OjBWieLIiv9W%2FPe6N3IBrtG9KF5rq%2Bx0VJ%2BxGBPqP4lieOQfycplzwV7DGFt2PGBXKyoiGPTKQwL1AfzjLd7xAp5S290flTHrerHFV%2BqgeoUUfNtKF151esmCRDboW5fnMTMOEbRuiYn1oOyXGRS8mQSRgYXAXEasguXjZCSbAwUyDLhzjOT1hkI%2F3cxzd%2BFPir2ayhtoRpnxxWvM3ES%2Br42minqfY1TfNvW0Eii5A4ErIpfbXRxrsfTbCyCMffFkw7TbbzEA7L2v3euwL4uvoXp65WqjbSDWfefSTqNQfR9z2ddOGHthRZmAq4FAl664gtNfYVbYXTDkzxZG7M7WmvOlFSo6G6teFWhhY2ip6C1XsVxjcIYVCoDJ204Xry26PM0wkvwI0%2F9xyPelOI8Qoeyk%2Bkiz28g1nRLxKTp6yKq9HBYoBy40WMh9kDICrCs4ncfBepei16iw%2F%2Ffm9OErQxNYeGvj5peWwEkMrzVSpcxsGkL%2B9wX4ltDOHRUbFh5sbp22yxpGASXqrhTf6dWutKgMI7F1zHX7jMMfRAjA%2BtMe2Ty1W6g8c2r7ABHsWL5z%2BZBwW%2BUXpQ3e674dZ7sTFWbXL1VFtjmsUE6qOoenwAICMw5ReyhZyOenxCbao5kmtjRk%2BH%2FRX8M3KS4CmLv596XGHTi1PDJMFY8NkCu8G9ZtsfdWvfmhJxKh6obm6TxIlRHlF9yyMsDKDMyPZIh3Ff9RxKGjELvN2vYRItthXtzOHs6VKeJicQL2fLQJFVKGomlXQHD%2BAdINO1CmluP1dr5b%2FZl5fMXNQHleXhSu0g3cF7RNc1yGDtcDPzrRhEqueNZDGgDh5UiOjvuP5d7io3%2Ft0SyYOu5F1raApkyZI5PPqV%2FdZjW%2FtHAJcLmf%2FEpfSVwChoQPISUS3rLHutNxI7LbGTP6s%2BgO2OhtZxdYNgvjew%3D%3D&ddcourse=BE&ddlbrcode=01&ddsem=Sem&ddl_effFrom=Academic%20Year&txtsubcode=&txtsubjectname=&btn_search=Search'
headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'en-US,en;q=0.9,gu-IN;q=0.8,gu;q=0.7,hi-IN;q=0.6,hi;q=0.5',
  'cache-control': 'no-cache',
  'content-type': 'application/x-www-form-urlencoded',
  'cookie': '_ga=GA1.3.1927007992.1719337760; _ga_YRXHKDQNVT=GS1.3.1722430732.3.0.1722430732.0.0.0; ASP.NET_SessionId=iivjojifa5u5numcvs0nrj1w; mp_844407e91b119d6a00359efeff26eaa4_mixpanel=%7B%22distinct_id%22%3A%20%22%24device%3A191c144bc8c552-0932e4ddd38a2b-10462c6f-144000-191c144bc8c552%22%2C%22%24device_id%22%3A%20%22191c144bc8c552-0932e4ddd38a2b-10462c6f-144000-191c144bc8c552%22%2C%22%24initial_referrer%22%3A%20%22%24direct%22%2C%22%24initial_referring_domain%22%3A%20%22%24direct%22%2C%22__mps%22%3A%20%7B%7D%2C%22__mpso%22%3A%20%7B%22%24initial_referrer%22%3A%20%22%24direct%22%2C%22%24initial_referring_domain%22%3A%20%22%24direct%22%7D%2C%22__mpus%22%3A%20%7B%7D%2C%22__mpa%22%3A%20%7B%7D%2C%22__mpu%22%3A%20%7B%7D%2C%22__mpr%22%3A%20%5B%5D%2C%22__mpap%22%3A%20%5B%5D%2C%22%24search_engine%22%3A%20%22google%22%7D',
  'origin': 'https://syllabus.gtu.ac.in',
  'pragma': 'no-cache',
  'priority': 'u=0, i',
  'referer': 'https://syllabus.gtu.ac.in/Syllabus.aspx?tp=BE',
  'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Linux"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [22]:
from bs4 import BeautifulSoup
html_content = response.text

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table by its ID
table = soup.find('table', {'id': 'GridViewToCategory'})
print(table)

# Initialize the dictionary to store the table data
table_data = {}

# Extract the headers (th elements) to use as dictionary keys
headers = [th.get_text(strip=True) for th in table.find('tr', class_='tbhedare').find_all('th')]

headers.remove('Exp.')
# Initialize each header in the dictionary with an empty list
for header in headers:
    table_data[header] = []

# Extract all data rows (tr elements) except the header
for row in table.find_all('tr')[1:]:
    print(row)
    cells = row.find_all('td')
    # Skip the first cell (expanding section)
    for i, cell in enumerate(cells[1:]):  # Start from the second cell
        cell_data = cell.get_text(strip=True)
        table_data[headers[i + 1]].append(cell_data)  # +1 to align with headers

<table cellspacing="0" class="myGrid" id="GridViewToCategory" rules="all" style="border-width:1px;border-style:solid;font-family:Arial;width:1050px;border-collapse:collapse;">
<tr class="tbhedare" style="color:White;">
<th scope="col">Exp.</th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$subcode')" style="color:White;">Subcode</a></th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$branch')" style="color:White;">Branch code</a></th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$with_eff_from')" style="color:White;">Eff_from</a></th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$subjectName')" style="color:White;">SubjectName</a></th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$category')" style="color:White;">Category</a></th><th scope="col"><a href="javascript:__doPostBack('GridViewToCategory','Sort$thCredit')" style="color:White;">Sem /Ye